<a href="https://colab.research.google.com/github/daeunni/Statistical_DL/blob/main/lec_code/Recurrent%20with%20IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
max_features=10000   # 10000개의 word를 가지고 하겠다
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)
print(type(x_train));print(type(x_test))
print(type(y_train));print(type(y_test))
print(x_train.shape)  # 25000개의 데이터
print(x_test.shape)
print(y_train.shape)

Using TensorFlow backend.


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)
(25000,)


In [ ]:
unique,count=np.unique(y_train,return_counts=True)   # 딕셔너리 형태로 unique와 count가 나온다
print(dict(zip(unique,count)))  # zip으로 매핑시켜서 출력한다
unique,count=np.unique(y_test,return_counts=True)
print(dict(zip(unique,count)))   ### stratified sampling을 한 것임

{0: 12500, 1: 12500}
{0: 12500, 1: 12500}


In [ ]:
print(x_train[0])
print(len(x_train[0]))
wordindex=imdb.get_word_index()
print(len(wordindex))  # 218개의 length
print(type(wordindex))
for key,value in wordindex.items():
    if value==1500:
        print(key)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
218
88584
<class 'dict'>
lacks


In [ ]:
exchindex=dict([(value,key) for (key,value) in wordindex.items()])
review=''.join([exchindex.get(i-3,'!') for i in x_train[0]])  # join으로 떨어진 것들이 다 붙음
print(review)

!thisfilmwasjustbrilliantcastinglocationscenerystorydirectioneveryone'sreallysuitedtheparttheyplayedandyoucouldjustimaginebeingthererobert!isanamazingactorandnowthesamebeingdirector!fathercamefromthesamescottishislandasmyselfsoilovedthefacttherewasarealconnectionwiththisfilmthewittyremarksthroughoutthefilmweregreatitwasjustbrilliantsomuchthatiboughtthefilmassoonasitwasreleasedfor!andwouldrecommendittoeveryonetowatchandtheflyfishingwasamazingreallycriedattheenditwassosadandyouknowwhattheysayifyoucryatafilmitmusthavebeengoodandthisdefinitelywasalso!tothetwolittleboy'sthatplayedthe!ofnormanandpaultheywerejustbrilliantchildrenareoftenleftoutofthe!listithinkbecausethestarsthatplaythemallgrownuparesuchabigprofileforthewholefilmbutthesechildrenareamazingandshouldbepraisedforwhattheyhavedonedon'tyouthinkthewholestorywassolovelybecauseitwastrueandwassomeone'slifeafterallthatwassharedwithusall


In [ ]:
from tensorflow.keras.preprocessing import sequence
lenmax=700
input_train=sequence.pad_sequences(x_train,maxlen=lenmax)  # pad_sequence를 진행(700개가 넘는건 잘라줌)
input_test=sequence.pad_sequences(x_test,maxlen=lenmax) 
print(input_train.shape)
print(input_test.shape)

(25000, 700)
(25000, 700)


In [ ]:
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.models import Sequential
srnn=Sequential()
srnn.add(Embedding(max_features,32, input_shape=700))  # 임베딩 (input length는 줘도되고 안줘도됨)
## 700개가 된 것을 linear combination을 취함. 
## 32 dimension이 들어옴 (dim32가 xt가 됨)
srnn.add(SimpleRNN(32))  # 32개의 x가 들어오고 이거의 linear combination을 취함
srnn.add(Dense(1,activation='sigmoid'))  ### 파라미터 계산할 수 있어야함!!!
srnn.summary()

srnn.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
srnn.fit(input_train,y_train,epochs=10,batch_size=128)
train_loss_acc=srnn.evaluate(input_train,y_train)
test_loss_acc=srnn.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)  # overfitting이 된 모습

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


C:\Users\ysp\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 21s 851us/step - loss: 0.5341 - acc: 0.7275
Epoch 2/10
25000/25000 [==============================] - 21s 859us/step - loss: 0.3490 - acc: 0.8575
Epoch 3/10
25000/25000 [==============================] - 21s 857us/step - loss: 0.2900 - acc: 0.8850
Epoch 4/10
25000/25000 [==============================] - 21s 857us/step - loss: 0.2506 - acc: 0.9034
Epoch 5/10
25000/25000 [==============================] - 21s 853us/step - loss: 0.2241 - acc: 0.9154
Epoch 6/10
25000/25000 [==============================] - 21s 857us/step - loss: 0.1934 - acc: 0.9275
Epoch 7/10
25000/25000 [==============================] - 22s 863us/step - loss: 0.1715 - acc: 0.9380
Epoch 8/10
25000/25000 [==============================] - 22s 862us/step - loss: 0.1449 - acc: 0.9477
Epoch 9/10
25000/25000 [==============================] - 22s 861us/step - loss: 0.1191 - acc: 0.9572
Epoch 10/10
25000/25000 [==============================] - 14s 556us/step
[0.14247

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
m_lstm=Sequential()
m_lstm.add(Embedding(max_features,32)) 
m_lstm.add(LSTM(32))   # 2080의 4배
m_lstm.add(Dense(1,activation='sigmoid'))
m_lstm.summary()
m_lstm.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
m_lstm.fit(input_train,y_train,epochs=10,batch_size=128)
train_loss_acc=m_lstm.evaluate(input_train,y_train)
test_loss_acc=m_lstm.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


C:\Users\ysp\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.4815 - acc: 0.7818
Epoch 2/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2880 - acc: 0.8852
Epoch 3/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2350 - acc: 0.9118
Epoch 4/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2021 - acc: 0.9251
Epoch 5/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1841 - acc: 0.9311
Epoch 6/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1635 - acc: 0.9416
Epoch 7/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1539 - acc: 0.9459
Epoch 8/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1415 - acc: 0.9509
Epoch 9/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1337 - acc: 0.9521
Epoch 10/10
25000/25000 [==============================] - 28s 1ms/step
[0.10286183683037758, 0.9681

In [ ]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
m_gru=Sequential()
m_gru.add(Embedding(max_features,32))
m_gru.add(GRU(32))  # 파라미터는 simple rnn의 3배
m_gru.add(Dense(1,activation='sigmoid'))
m_gru.summary()
m_gru.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
m_gru.fit(input_train,y_train,epochs=10,batch_size=128)
train_loss_acc=m_gru.evaluate(input_train,y_train)
test_loss_acc=m_gru.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 326,273
Trainable params: 326,273
Non-trainable params: 0
_________________________________________________________________


C:\Users\ysp\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.4769 - acc: 0.7598
Epoch 2/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.2899 - acc: 0.8812
Epoch 3/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.2409 - acc: 0.9069
Epoch 4/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.2116 - acc: 0.9203
Epoch 5/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.1935 - acc: 0.9271
Epoch 6/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.1757 - acc: 0.9360
Epoch 7/10
25000/25000 [==============================] - 64s 3ms/step - loss: 0.1625 - acc: 0.9409
Epoch 8/10
25000/25000 [==============================] - 66s 3ms/step - loss: 0.1479 - acc: 0.9474
Epoch 9/10
25000/25000 [==============================] - 66s 3ms/step - loss: 0.1432 - acc: 0.9496
Epoch 10/10
25000/25000 [==============================] - 31s 1ms/step
[0.09908104753017426, 0.9683

In [ ]:
from tensorflow.keras.layers import GRU, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
model=Sequential()
model.add(Embedding(max_features,32))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.5, return_sequences=True))  # input은 h1부터 h700까지(return sequence를 True로 설정)
# dropout은 lstm의 2개의 인풋 중 xt의 dropout을 시키는거고, recurrent_dropout은 ct(상태벡터)를 dropout시킴(ht를 dropout시키는게 아님)
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5))  # 700*32 = 64 combination이 됨
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.fit(input_train,y_train,epochs=10,batch_size=128)

#### 파라미터 개수 계산하는거 확인하기!!! 2장이랑 살펴보면서

train_loss_acc=model.evaluate(input_train,y_train)
test_loss_acc=model.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)  # 시간이 오래걸림,,

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 32)          320000    
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 353,217
Trainable params: 353,217
Non-trainable params: 0
_________________________________________________________________


C:\Users\ysp\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 221s 9ms/step - loss: 0.6021 - acc: 0.6617
Epoch 2/10
25000/25000 [==============================] - 235s 9ms/step - loss: 0.4202 - acc: 0.8203
Epoch 3/10
25000/25000 [==============================] - 241s 10ms/step - loss: 0.3588 - acc: 0.8493
Epoch 4/10
25000/25000 [==============================] - 243s 10ms/step - loss: 0.3242 - acc: 0.8664
Epoch 5/10
25000/25000 [==============================] - 242s 10ms/step - loss: 0.3042 - acc: 0.8780
Epoch 6/10
25000/25000 [==============================] - 241s 10ms/step - loss: 0.2836 - acc: 0.8891
Epoch 7/10
25000/25000 [==============================] - 242s 10ms/step - loss: 0.2680 - acc: 0.8956
Epoch 8/10
25000/25000 [==============================] - 242s 10ms/step - loss: 0.2574 - acc: 0.9024
Epoch 9/10
25000/25000 [==============================] - 243s 10ms/step - loss: 0.2444 - acc: 0.9072
Epoch 10/10
25000/25000 [==============================] - 167s 7ms/step
[0.19449921